<a href="https://colab.research.google.com/github/mansh2060/QA-bot/blob/main/QAbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import wget
from transformers import DPRContextEncoder,DPRContextEncoderTokenizer,DPRQuestionEncoder,DPRQuestionEncoderTokenizer
import torch
import numpy as np
from transformers import AutoTokenizer,AutoModelForCausalLM
from pinecone import Pinecone,ServerlessSpec,PodSpec

In [ ]:
filename='companypolicies.txt'
url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt'
wget.download(url,out=filename)

'companypolicies (1).txt'

In [ ]:
#file-read + data - list + data -space eliminate  = return paragraph
def read_file(filename):
    with open(filename,'r',encoding='utf-8') as files:
        file=files.read()

    paragraphs=file.split('\n')
    paragraphs=[para.strip() for para in paragraphs if len(para.strip()) > 0 ]
    return paragraphs
paragraphs=read_file('companypolicies.txt')

In [ ]:
model_name="facebook/dpr-ctx_encoder-single-nq-base"

Used DPR for tokenization,embeddings,question answer answering instead of OPENAI API

In [ ]:
def encode_contexts(text_list):
    embeddings=[]
    context_tokenizer=DPRContextEncoderTokenizer.from_pretrained(model_name)
    context_encoder=DPRContextEncoder.from_pretrained(model_name)
    for text in text_list:
        tokens=context_tokenizer(text,padding=True,return_tensors="pt",truncation=True,max_length=256)
        vectors=context_encoder(**tokens)
        embeddings.append(vectors.pooler_output)
    return torch.cat(embeddings).detach().numpy()
context_embedding=encode_contexts(paragraphs)


In [ ]:
len(context_embedding)

76

In [ ]:
context_embedding.shape

(76, 768)

In [ ]:
context_embedding

array([[ 1.2332821 , -1.1841292 ,  0.03847372, ...,  0.24722798,
         1.3102813 , -1.9066025 ],
       [ 0.97840405, -1.019581  ,  0.24952444, ...,  0.41552603,
         1.2107834 , -1.6416715 ],
       [ 0.9553924 , -1.1559378 ,  0.2559863 , ...,  0.48858047,
         1.2516516 , -1.7455238 ],
       ...,
       [ 0.7808161 , -1.0014777 ,  0.32689705, ...,  0.27574834,
         1.1540724 , -1.6400774 ],
       [ 0.88111687, -0.99578106,  0.20064867, ...,  0.51199406,
         1.2861305 , -1.7275058 ],
       [ 0.84801227, -0.9993715 ,  0.35512507, ...,  0.31374866,
         1.4298874 , -1.6731874 ]], dtype=float32)

Used PineCone for vector database

In [ ]:
import os
use_serverless=True

In [ ]:
api_key=os.environ.get('PINECONE_API_KEY')
pc=Pinecone(api_key="pcsk_3LLvfq_RmMBF385Va33QedCqyG6cU9HuvANvMbXnLXYzzP25wK4Lmyx8jRadnkUQ4tuHWN")

In [ ]:
if use_serverless:
  spec=ServerlessSpec(cloud='aws',region='us-east-1')
else:
  spec=PodSpec(environment='environment')

In [ ]:
index_name="ragdatabase"
if index_name in pc.list_indexes().names():
   pc.delete_index(index_name)

In [ ]:
index_name = "ragdatabase"
if index_name not in pc.list_indexes():
    pc.create_index(index_name,
                    dimension=768,
                    metric="cosine",
                    spec=ServerlessSpec(
                    cloud='aws',
                    region='us-east-1'
                     )
                    )
index = pc.Index(index_name)
ids = [f"id-{i}" for i in range(len(context_embedding))]

data = [(ids[i], context_embedding[i].tolist()) for i in range(len(context_embedding))]
index.upsert(vectors=data)
print("Vectors uploaded successfully to Pinecone!")


Vectors uploaded successfully to Pinecone!


In [ ]:
question_encoder=DPRQuestionEncoder.from_pretrained(model_name)
question_tokenizer=DPRQuestionEncoderTokenizer.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
model.generation_config.pad_token_id = tokenizer.pad_token_id

In [255]:
def generate_answer(question):
    inputs = tokenizer(question, return_tensors='pt', max_length=1024, truncation=True)

    summary_ids = model.generate(
        inputs['input_ids'],
        max_length=150,  # Limit the answer length
        min_length=40,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=3,  # Avoid repeating n-grams of size 3
        temperature=0.7,  # Control the randomness of the output
        pad_token_id=tokenizer.eos_token_id
    )

    answer = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return answer


# Testing ***QA*** Bot from generate answer function

In [256]:
question = "what is Smoking policy?"
answer = generate_answer(question)
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


what is Smoking policy?

Smoking is defined as "the use of tobacco products, including cigarettes, cigars, chewing tobacco, chewing gum, and chewing gum products, in a manner that is harmful to the health or welfare of any person or group of persons."

What is the difference between smoking and smoking-related diseases? What are the health risks of smoking? What is the role of smoking in the prevention of diseases such as cancer, heart disease, stroke, diabetes, and cancer of the pancreas, liver, and lung? What does smoking have to do with smoking? How does smoking affect the health of the human body? How is smoking different from other forms of tobacco use? Is smoking harmful to human health?



In [257]:
question = "what is Recruitment policy?"
answer = generate_answer(question)
print(answer)

what is Recruitment policy?

Recruitment is the process by which a recruiter selects a candidate for a job. Recruiters use this process to select a candidate based on a number of factors such as age, gender, education level, and experience.

For example, if you are looking for someone with a bachelor's degree, you may want to look for someone who has a Bachelor's degree. If you want someone with an M.B.A., you may also want someone who is a graduate of a college or university and has a Ph.D. In addition, you can look for people who have been in the workforce for at least 10 years and who have had at least one year of experience in the


In [258]:
question = "what is Internet and Email policy?"
answer = generate_answer(question)
print(answer)

what is Internet and Email policy?

Internet and email policies apply to all of our products and services. If you have any questions about our policies, please contact us.

How do I register for a free trial of my product?
.
. You can register for free at any time by clicking on the "Register Now" button on the right hand side of this page. You will be asked to provide your email address and password. You may also choose to sign up for a trial of your product by clicking the "Sign Up" button at the bottom of the page. Once you have signed up for your trial, you will be able to download and install the free version of the product. You do not need to register for


In [259]:
question = "what is Drug and Alcohol policy?"
answer = generate_answer(question)
print(answer)

what is Drug and Alcohol policy?

Drug and alcohol policy is an important part of the federal government's approach to drug policy. The Drug Enforcement Administration (DEA) is responsible for enforcing the Controlled Substances Act (CSA) and the Controlled Substance Act (SAMHSA), both of which are federal laws. The DEA is also responsible for administering the National Institute on Drug Abuse (NIDA), which is the federal agency that administers the National Institutes of Health (NIH). The NIDA is a federally funded, non-profit organization dedicated to the prevention, treatment, and education of drug and alcohol abuse.

The NIDA's mission is to promote the health and well-being of the American people through education, research


In [260]:
question = "what is Health and Safety Policy?"
answer = generate_answer(question)
print(answer)

what is Health and Safety Policy?

Health and safety policy is a set of policies that govern the health and safety of individuals, businesses, and organizations. These policies are designed to ensure that individuals and businesses comply with the laws, regulations, and regulations of the federal government.

What are the different types of health care policies that are covered by the Affordable Care Act (ACA)?

The ACA is a health care law that provides health care coverage to all Americans. The ACA is designed to provide coverage for all Americans, regardless of income, race, color, religion, national origin, age, disability, marital status, or any other factor that affects the health of the individual or business. The health care plan offered under the ACA is
